In [1]:
import numpy as np
import pandas as pd

import os
import re


from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, BertModel, AdamW, get_linear_schedule_with_warmup, BertForSequenceClassification,TrainingArguments,Trainer, AutoTokenizer
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

c:\Users\mateu\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CSV_PATH = "C:\TCC\Base\merged_data.csv" 

# Carregar dados do CSV usando pandas
data = pd.read_csv(CSV_PATH, sep='|', usecols=["text","category","label"])
data["text"] = data["text"].str.replace('\n', ' ').replace('\t', ' ').replace('   ', ' ').replace('  ', ' ')
data = shuffle(data).reset_index(drop=True)

original_label_dtype = data['label'].dtype
data['label'] = data['label'].astype(str)

data['stratify_col'] = data['category'] + '-' + data['label']

data.drop('category', axis=1, inplace=True)
data['label'] = data['label'].astype(original_label_dtype)

#(60%)(20%)(20%)
train_data, temp_data = train_test_split(data, test_size=0.4, stratify=data['stratify_col'], random_state=42)
validate_data, test_data = train_test_split(temp_data, test_size=0.5, stratify=temp_data['stratify_col'], random_state=42)

train_data.drop('stratify_col', axis=1, inplace=True)
validate_data.drop('stratify_col', axis=1, inplace=True)
test_data.drop('stratify_col', axis=1, inplace=True)

print("Training size: {}".format(len(train_data)))
print("Validation size: {}".format(len(validate_data)))
print("Testing size: {}".format(len(test_data)))

display(train_data)
display(validate_data)
display(test_data)

Training size: 4200
Validation size: 1400
Testing size: 1400


,text,label
3961,URGENTE: Palácio do Alvorada é isolado. Há rel...,1
1010,"Deputado desabafa: ""Governo lixo! Liberou um c...",1
5395,"Trump defende imigração por mérito, promete re...",0
6416,Ministro admite que governo poderá recorrer à ...,0
1840,A realidade é pior que você imagina: a jornali...,0
...,...,...
2113,Julgamento do recurso de Lula no TRF-4 ocorre ...,0
6739,Coreia do Norte acusa CIA de complô para assas...,0
759,Por que o populismo está crescendo na direita ...,0
6243,Ministra do PT viaja para a China e fecha acor...,1


,text,label
5621,"""Estou convencido de que ele me gravou"" disse ...",1
6842,"Pitaia, sardinhas, quiosques badalados, sereia...",0
2224,"De assédio à religião, as dicas de mulheres qu...",0
499,PT investiu R$ 50 bi em obras no exterior e de...,1
72,Ex-presidente diz que é mais honesto do que to...,1
...,...,...
6905,O cientista político capixaba Bruno Garschagen...,0
1529,Sensitiva famosa emociona a internet e diz que...,1
6084,"Presente da ""mamãe"" para o povo: Governo vai s...",1
5497,"Voar, cair, voar de novo. No movimento de expa...",0


,text,label
6286,"Em decisão unânime, a 8.a Turma do Tribunal Re...",0
5917,Checava se alguém se mataria ao vivo: a rotina...,0
148,Impeachment no senado: Se a votação fosse hoje...,1
2904,Veja ponto a ponto o que diz a defesa de Miche...,0
3202,"Boa Esporte emite nota: ""Não cometemos nenhum ...",1
...,...,...
3794,Veja detalhes da operação que encontrou digita...,0
2660,‘Sou liberal. Centro é a posição mais equili...,0
4504,"Nessa semana, fui apresentado a mais um sistem...",0
1687,O operador de propinas Fernando Antonio Falcão...,0


In [3]:

train_data = train_data.reset_index(drop=True)
validate_data = validate_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [4]:
print("Training size: {}".format(len(train_data)))
print("Validation size: {}".format(len(validate_data)))
print("Testing size: {}".format(len(test_data)))

Training size: 4200
Validation size: 1400
Testing size: 1400


In [5]:
#Escolhendo a GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [6]:
# tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = BertForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", from_tf=True)
model.config.num_labels = 1
#load no Tokenizador e o modelo BERTimbau

All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


In [7]:
# #Desabilitando a atualização dos gradientes para acelerar o processamento
# for param in model.parameters():
#     param.requires_grad = False

In [8]:
# Definindo a arquitetura da camada classificadora
# composta por várias camadas densas e de ativação para processamento não linear e uma camada Softmax 
# para obter as probabilidades de cada classe.
model.classifier = nn.Sequential(
    nn.Linear(768, 256),
    nn.ReLU(),
    nn.Linear(256, 64),
    nn.ReLU(),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)
)

In [9]:
model = model.to(device)

# Alocando o modelo e o critério de perda na GPU
criterion = nn.MSELoss().to(device)

# Atualizar apenas os parâmetros da camada classificadora com uma taxa de aprendizado de 0,01.
optimizer = optim.SGD(model.classifier.parameters(), lr=0.01)

In [10]:
def clean_text(text):
  #limpando String
  return text.replace('\n', ' ').replace('\t', ' ').replace('   ', ' ').replace('  ', ' ')
 

def preprocess_text(text):
    text = clean_text(text)
    parts = []

    text_len = len(text.split(' '))
    delta = 300 #numero de palavras
    max_parts = 5 #numero maximo de partes a serem divididas
    nb_cuts = int(text_len / delta)
    nb_cuts = min(nb_cuts, max_parts)
    
    #Tokenizando o texto
    for i in range(nb_cuts + 1):
        text_part = ' '.join(text.split(' ')[i * delta: (i + 1) * delta])
        parts.append(tokenizer.encode(text_part, return_tensors="pt", max_length=500).to(device))

    return parts

def testText(text):
    text_parts = preprocess_text(text)
    overall_output = torch.zeros((1,2)).to(device)
    try:
        for part in text_parts:
            if len(part) > 0:
                overall_output += model(part.reshape(1, -1))[0]
    except RuntimeError:
        print("GPU out of memory, skipping this entry.")

    overall_output = F.softmax(overall_output[0], dim=-1)

    value, result = overall_output.max(0)

    term = "falso"
    if result.item() == 0:
        term = "verdadeiro"

    print("{} : {}%".format(term, value.item() * 100))
    return term

In [11]:
class BertFakeNewsDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_seq_length=512):
        self._labels = torch.tensor(df["label"].values, dtype=torch.long)
        self._encodings = {"input_ids": [],
                           "token_type_ids": [],
                           "attention_mask": []}

        for txt in df["text"].values:
            enc_dict = tokenizer.encode_plus(
                text=txt,
                add_special_tokens=True,
                max_length=max_seq_length,
                return_token_type_ids=True,
                padding="max_length",
                return_attention_mask=True,
                return_tensors="pt",
                truncation=True,
            )
            # Verifica se a parte tokenizada excede o limite após adicionar tokens especiais
            if len(enc_dict["input_ids"][0]) > max_seq_length:
                print("Truncamento detectado após adição de tokens especiais!")

            for k, v in enc_dict.items():
                self._encodings[k].append(v[0])
        
    def __getitem__(self, idx):
        item = {key: value[idx] for key, value in self._encodings.items()}
        item["labels"] = self._labels[idx]
        return item

    def __len__(self):
        return len(self._labels)

In [12]:
train_dataTK = BertFakeNewsDataset(train_data,tokenizer)
validate_dataTK = BertFakeNewsDataset(validate_data,tokenizer)
test_dataTK = BertFakeNewsDataset(test_data,tokenizer)

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "qwk": cohen_kappa_score(labels, predictions, weights="quadratic"),
    }

In [14]:
train_args = TrainingArguments(
    output_dir='C:\TCC',
    evaluation_strategy="epoch",
    learning_rate=2.5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=1e-5, #1e-5 ou 5e-5 é um valor padrão 0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_strategy="epoch",
    save_steps=1000,
    save_total_limit=10,
)

In [15]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_dataTK,
    eval_dataset=validate_dataTK,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [16]:
torch.cuda.empty_cache()

In [17]:
trainer.train()

c:\Users\mateu\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/2630 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 8.00 GiB total capacity; 2.58 GiB already allocated; 3.66 GiB free; 2.62 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.evaluate(train_dataTK)

In [ ]:
trainer.evaluate(validate_dataTK)

In [ ]:
trainer.evaluate(test_dataTK) 

In [ ]:
model = trainer.model

In [ ]:
falso = """Mais uma pérola da "senhora que estoca vento". Em nota, ela sugere que a PF "tortura" investigados.  A ex-presidente impeachmada Dilma Rousseff continua falando pelos cotovelos.  (observação: Em seu texto, ela não usou a palavra "tortura" [...] mas para quem sabe ler, um pingo é letra!) Durante a última semana, Dilma teve a audácia (a petulância, a ousadia, destemor) de dizer que os investigadores da Polícia Federal estão ameaçando os delatores da Lava-Jato. Dilma também atacou o ministro relator da Lava-Jato, Edson Fachin, e o acusou de abrir as delações de João Santana e Mônica Moura para a imprensa antes sem antes dar acesso dos documentos a seus advogados. Resumindo, Dilma é honesta [...] já Edson Fachin (Ministro do STF) e a Polícia Federal são os "bandidos da história" Em uma nota divulgada na última sexta-feira, a "senhora que estoca vento" também disse que todos os delatores são mentirosos. abaixo a íntegra da nota: 1. Infelizmente, chega tarde a decisão do relator da Lava Jato no Supremo Tribunal Federal, ministro Edson Fachin, suspendendo o sigilo dos depoimentos de João Santana e Monica Moura. 2. Há semanas, a defesa da presidenta eleita Dilma Rousseff havia feito tal pedido ao Tribunal Superior Eleitoral, a fim de apresentar suas alegações finais ao relator do caso das contas de campanha, ministro Herman Benjamin. 3. A defesa foi prejudicada pela negativa do relator. Não foi possível cotejar os depoimentos prestados pelo casal à Justiça Eleitoral e na Lava Jato. 4. As contradições e falsos testemunhos foram vislumbrados, apesar disso, pelo que foi divulgado amplamente pela imprensa, na velha estratégia do vazamento seletivo dos depoimentos – uma rotina nos últimos tempos. 5. Agora mesmo, os depoimentos são entregues à imprensa, mas não repassados oficialmente à defesa da presidente eleita. 6. Dilma Rousseff, contudo, reitera o que apontou antes: João Santana e Monica Moura prestaram falso testemunho e faltaram com a verdade em seus depoimentos, provavelmente pressionados pelas ameaças dos investigadores. 7. Apesar de tudo, a presidente eleita acredita na Justiça e sabe que a verdade virá à tona e será restabelecida."""

Verdadeiro = """Saúde de Lula é 'excelente', atestam exames feitos hoje

O ex-presidente Luiz Inácio Lula da Silva encontra-se em "excelente condição de saúde e sem qualquer evidência de neoplasia", diz o boletim assinado pelos médicos Antonio Carlos Onofre de Lira, diretor técnico médico, e Paulo Cesar Ayroza Galvão, diretor clínico do hospital sírio-libanês.

Lula foi submetido a exames clínicos, laboratoriais, PET-CT, ressonância nuclear magnética e laringoscopia durante toda a manhã deste sábado, 1. Todos de rotina. Essa rotina vem sendo repetida desde 2011, quando foi detectado um câncer na laringe do ex-presidente, que se submeteu a tratamento e deve fazer o controle por cinco anos. O último exame havia sido realizado em 10 de agosto. As equipes que acompanham o ex-presidente são coordenadas pelos médicos Roberto Kalil Filho e Artur Katz. Lula deixou o hospital sem falar com a imprensa.
"""

testText(falso)
testText(Verdadeiro)

In [ ]:
from sklearn.metrics import f1_score

# Obter as previsões do modelo para o conjunto de teste
predictions = trainer.predict(test_dataTK).predictions
predicted_labels = np.argmax(predictions, axis=1)

# Obter os rótulos verdadeiros do conjunto de teste
true_labels = test_dataTK._labels.numpy()

# Calcular a métrica F1
f1 = f1_score(true_labels, predicted_labels)

print("F1 Score:", f1)